In [1]:
import os
import re
import sys
import ssl
import json
import nltk
import time
import math
import string
import pickle
import argparse

import torch
import numpy as np
import pandas as pd
from torch import optim
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as tud
from collections import Counter, OrderedDict
from sklearn.model_selection import train_test_split

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, LancasterStemmer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from torch.nn.utils.rnn import pad_sequence, pack_sequence, pad_packed_sequence, pack_padded_sequence

In [2]:
# trn = pd.read_pickle('jigsaw_toxic/20pct_model_35toxic_train.pkl')
# val = pd.read_pickle('jigsaw_toxic/20pct_model_35toxic_val.pkl')

In [2]:
import jigsaw_toxic.exec_lstm as exl

Using CUDA? True cuda:0


In [135]:
torch.cuda.is_available()

True

In [5]:
# X_train = trn.drop('label', axis=1)
# y_train = trn['label']
# test_sample = val.sample( n=math.ceil(len(X_train)*0.2), random_state=1008 )
# X_test = test_sample.drop('label', axis=1)
# y_test = test_sample['label'] 

In [6]:
# lstm_model = exl.LSTMModel(X_train[:500], y_train[:500], X_test[:100], y_test[:100], 
#                            hidden_dim=50, num_layers=1, embed_dim=50, batch_size=1,
#                            dropout=0, num_classes=2)

In [11]:
# if exl.USE_CUDA:
#     print(exl.USE_CUDA)
#     lstm_model = lstm_model.cuda()

In [8]:
# lstm_model.train()

LSTMModel(
  (loss_fcn): NLLLoss()
  (embedding): Embedding(7500, 50)
  (lstm): LSTM(50, 50, batch_first=True)
  (linear): Linear(in_features=50, out_features=2, bias=True)
)

In [9]:
# NUM_EPOCHS = 6
# hist_lstm = np.zeros(NUM_EPOCHS)
# model_name="toy_dataset"

In [10]:
# results, model_state_dict = lstm_model.run_model(y_train[:500], X_test[:100], y_test[:100], NUM_EPOCHS, 
#                                                  hist_lstm, text_col='cleaned_no_stem', savestate=model_name) 

Epochs: 6; Train Size: 500; Test Size: 100)
Starting epoch 0...
Epoch 0, Negative Log Linear Loss: 0.38605058193206787

Starting Evaluation


/home/lhinkson/.local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/lhinkson/.local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Epoch 0 F1 Score: 0.0
Starting epoch 1...

Starting Evaluation
Epoch 1 F1 Score: 0.0
Starting epoch 2...
Epoch 2, Negative Log Linear Loss: 0.3697086274623871

Starting Evaluation
Epoch 2 F1 Score: 0.0
Starting epoch 3...

Starting Evaluation
Epoch 3 F1 Score: 0.0
Starting epoch 4...
Epoch 4, Negative Log Linear Loss: 0.36790764331817627

Starting Evaluation
Epoch 4 F1 Score: 0.0
Starting epoch 5...

Starting Evaluation
Epoch 5 F1 Score: 0.0
Highest F1 Score: 0.0, Epoch: 0


In [11]:
# exl.get_metrics(results, should_print=True, detailed = False, 
#                 label_col="y_true", score_col="predicted_score")

Group Size: 100
Overall Accuracy: 0.89
Overall Precision: 0.0
Overall Recall: 0.0
Overall F1 Score: 0.0
ROC_AUC: 0.5

Group Size: 11
Target Accuracy: 0.0
Target Precision: 0.0
Target Recall: 0.0
Target F1 Score: 0.0

Group Size: 89
Non-Target Accuracy: 1.0
Non-Target Precision: 1.0
Non-Target Recall: 1.0
Non-Target F1 Score: 1.0



/home/lhinkson/.local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/lhinkson/.local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/lhinkson/.local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/lhinkson/.local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


{'Overall': {'Accuracy': 0.89,
  'Precision': 0.0,
  'Recall': 0.0,
  'F1': 0.0,
  'ROC_AUC': 0.5},
 'Target': {'Accuracy': 0.0, 'Precision': 0.0, 'Recall': 0.0, 'F1': 0.0},
 'Non-Target': {'Accuracy': 1.0, 'Precision': 1.0, 'Recall': 1.0, 'F1': 1.0}}

### Creating Batch Loader to Enable Larger Dataset Training

In [11]:
train_full = pd.read_csv("jigsaw_toxic/train.csv")

In [12]:
train_full['label'] = train_full.target.apply(lambda x: 1 if x > 0.5 else 0)

In [13]:
test_ratio = 0.2

In [14]:
train_sample, val_set, test_set = exl.get_samples(train_full, proportion=0.2, 
                                                  train_test_ratio=(1-test_ratio))

Retrieving training samples at proportion 0.2


In [15]:
print(train_sample.shape, val_set.shape, test_set.shape)

(288690, 46) (180325, 46) (181100, 46)


In [16]:
val_samples = val_set.sample( n=math.ceil(len(train_sample)*test_ratio), random_state=1008 )

In [17]:
val_samples.shape

(57738, 46)

In [55]:
train_sample[['comment_text', 'label']].to_csv("train_samples_060219.csv", index=False)
val_samples[['comment_text', 'label']].to_csv("val_samples_060219.csv", index=False)

In [146]:
#  Attribution: https://stackoverflow.com/questions/52602071/dataframe-as-datasource-in-torchtext
from torchtext.data import Field, Dataset, Example
import pandas as pd

class DataFrameDataset(Dataset):
    """Class for using pandas DataFrames as a datasource"""
    def __init__(self, examples, fields, filter_pred=None):
        """
        Create a dataset from a pandas dataframe of examples and Fields
        Arguments:
        examples pd.DataFrame: DataFrame of examples
            fields {str: Field}: The Fields to use in this tuple. The
            string is a field name, and the Field is the associated field.
        filter_pred (callable or None): use only exanples for which
            filter_pred(example) is true, or use all examples if None.
            Default is None
        """
        self.examples = examples.apply(SeriesExample.fromSeries, args=(fields,), axis=1).tolist()
        if filter_pred is not None:
            self.examples = filter(filter_pred, self.examples)
        self.fields = dict(fields)
        # Unpack field tuples
        for n, f in list(self.fields.items()):
            if isinstance(n, tuple):
                self.fields.update(zip(n, f))
                del self.fields[n]

                
class SeriesExample(Example):
    """Class to convert a pandas Series to an Example"""

    @classmethod
    def fromSeries(cls, data, fields):
        return cls.fromdict(data.to_dict(), fields)

    @classmethod
    def fromdict(cls, data, fields):
        ex = cls()

        for key, field in fields.items():
#             print(key)
#             print(field)
            if key not in data:
                raise ValueError("Specified key {} was not found in "
                "the input data".format(key))
            if field is not None:
                setattr(ex, key, field.preprocess(data[key]))
            else:
                setattr(ex, key, data[key])
        return ex

In [141]:
TEXT = Field(tokenize=tokenizer, lower=True, init_token = "<sos>", eos_token = "<eos>", batch_first=True)
LABEL = data.Field(sequential=False, unk_token=None, dtype=torch.float, is_target=True)

In [142]:
train_ds = DataFrameDataset(train_sample, fields = { 'label' : LABEL, 'comment_text' : TEXT })
valid_ds = DataFrameDataset(val_samples, fields = { 'label' : LABEL, 'comment_text' : TEXT })


In [133]:
train_ds

In [143]:
TEXT.build_vocab(train_ds, max_size=25000)
LABEL.build_vocab(train_ds)

In [144]:
print(TEXT.vocab.stoi)

defaultdict(<function _default_unk_index at 0x7f7d7015b6a8>, {'<unk>': 0, '<pad>': 1, '<sos>': 2, '<eos>': 3, 'i': 4, 'not': 5, 'the': 6, 'would': 7, 'people': 8, 'one': 9, 'like': 10, 'trump': 11, 'get': 12, 'them': 13, 'should': 14, 'us': 15, 'time': 16, 'think': 17, 'it': 18, 'many': 19, 'know': 20, 'even': 21, 'good': 22, 'you': 23, 'right': 24, 'if': 25, 'over': 26, 'years': 27, 'see': 28, 'much': 29, 'make': 30, 'and': 31, 'way': 32, 'need': 33, 'well': 34, 'could': 35, 'this': 36, 'government': 37, 'want': 38, 'money': 39, 'go': 40, 'state': 41, 'tax': 42, 'also': 43, 'going': 44, 'they': 45, 'never': 46, 'say': 47, 'take': 48, 'we': 49, 'really': 50, 'back': 51, 'said': 52, 'new': 53, "i'm": 54, 'he': 55, 'but': 56, 'what': 57, 'still': 58, 'pay': 59, 'canada': 60, 'work': 61, 'so': 62, 'first': 63, "it's": 64, "that's": 65, 'every': 66, 'year': 67, 'better': 68, 'world': 69, 'country': 70, 'nothing': 71, 'may': 72, 'no': 73, 'public': 74, 'another': 75, "can't": 76, 'that': 77

In [145]:
print(TEXT.vocab.itos[437])

provide


In [147]:
import torch
from torchtext import data
import numpy as np

global max_text_in_batch

def batch_size_fn(new, count, sofar):
    "Keep augmenting batch and calculate total number of tokens + padding."
    global max_text_in_batch
    if count == 1:
        max_text_in_batch = 0
    max_text_in_batch = max(max_text_in_batch, len(new.comment_text) + 2)
    text_elements = count * max_text_in_batch
    return text_elements


class EfficientIterator(data.Iterator):
    def create_batches(self):
        if self.train:
            def pool(d, random_shuffler):
                for p in data.batch(d, self.batch_size * 100):
                    p_batch = data.batch(
                        sorted(p, key=self.sort_key),
                        self.batch_size, self.batch_size_fn)
                    for b in random_shuffler(list(p_batch)):
                        yield b
            self.batches = pool(self.data(), self.random_shuffler)
            
        else:
            self.batches = []
            for b in data.batch(self.data(), self.batch_size,
                                          self.batch_size_fn):
                self.batches.append(sorted(b, key=self.sort_key))

In [114]:
# import spacy
# import re

# class tokenize(object):
    
#     def __init__(self, lang):
#         self.nlp = spacy.load(lang)
            
#     def tokenizer(self, sentence):
#         sentence = re.sub(
#         r"[\*\"“”''\n\\…\+\-\/\=\(\)‘•:\[\]\|’\!;]", " ", str(sentence))
#         sentence = re.sub(r"[ ]+", " ", sentence)
#         sentence = re.sub(r"\!+", "!", sentence)
#         sentence = re.sub(r"\,+", ",", sentence)
#         sentence = re.sub(r"\?+", "?", sentence)
#         sentence = sentence.lower()
#         return [tok.text for tok in self.nlp.tokenizer(sentence) if tok.text != " "]

In [16]:
# tok = tokenize('en_core_web_sm')

In [17]:
# tok.tokenizer(train_full['comment_text'].values[0])

['this',
 'is',
 'so',
 'cool',
 '.',
 'it',
 's',
 'like',
 ',',
 'would',
 'you',
 'want',
 'your',
 'mother',
 'to',
 'read',
 'this',
 '?',
 'really',
 'great',
 'idea',
 ',',
 'well',
 'done']

In [19]:
def tokenizer(text, stop_ws=exl.stops, stemmer=None, str_output=False):

    t = text.replace("-", " ").split(" ")
    t = [w.strip(string.punctuation) for w in t]

    if stop_ws:
        t = [w.lower() for w in t if w not in stop_ws]

    if stemmer:
        t = [stemmer.stem(w) for w in t]

    if str_output:
        return ' '.join(t)
    else:
        return t

In [21]:
# tokenizer(train_full['comment_text'].values[0])

['this',
 'cool',
 "it's",
 'like',
 'would',
 'want',
 'mother',
 'read',
 'really',
 'great',
 'idea',
 'well',
 'done']

In [124]:
from torchtext.data import Field, BucketIterator, TabularDataset

TEXT = Field(tokenize=tokenizer, lower=True, init_token = "<sos>", eos_token = "<eos>", batch_first=True)
LABEL = data.Field(sequential=False, unk_token=None, dtype=torch.float)



In [91]:
data_fields = [('comment_text', TEXT), ('label', LABEL)]
trn,val = data.TabularDataset.splits(path='./', train='train_samples_060219.csv', 
                                       validation='val_samples_060219.csv', format='csv', 
                                       fields=data_fields)

In [101]:
trn.fields

{'comment_text': <torchtext.data.field.Field at 0x7f7cd10979b0>,
 'label': <torchtext.data.field.Field at 0x7f7cd1097da0>}

In [94]:
TEXT.build_vocab(train_sample, max_size=20000)
LABEL.build_vocab(train_sample, val)

In [148]:
print(TEXT.vocab.stoi['leave'])

434


In [152]:
print(TEXT.vocab.itos[434])

leave


In [153]:
train_iter = BucketIterator(trn, batch_size=20, \
sort_key=lambda x: len(x.comment_text), shuffle=True)

In [154]:
batch = next(iter(train_iter))
print(batch.comment_text)

tensor([[    2,  2240,   358,  ...,     1,     1,     1],
        [    2,   131,   751,  ...,     1,     1,     1],
        [    2,    78,   432,  ...,     1,     1,     1],
        ...,
        [    2,     6, 10192,  ...,     1,     1,     1],
        [    2,   761,   364,  ...,     1,     1,     1],
        [    2,   991, 37545,  ...,     1,     1,     1]])


In [155]:
train_iter = EfficientIterator(
    trn, batch_size=1300, 
    device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
    repeat=False, sort_key= lambda x: len(x.comment_text), batch_size_fn=batch_size_fn, 
    train=True, shuffle=True)


In [156]:
batch = next(iter(train_iter))
print(batch.comment_text)

tensor([[    2,  1272,   206,  ...,   469,  4655,     3],
        [    2,    54,     5,  ...,    71,    47,     3],
        [    2,   733,    20,  ...,   282,  1524,     3],
        ...,
        [    2, 15249,   427,  ...,  1170,   741,     3],
        [    2,   467,   183,  ..., 27923, 41165,     3],
        [    2,    61,   393,  ...,   225, 36611,     3]], device='cuda:0')


In [32]:
from tqdm import tqdm

def make_examples(df, fields):
    fields = {field_name: (field_name, field)
              for field_name, field in fields.items()}
    for _, row in tqdm(df.iterrows()):
        example = data.Example.fromdict(row, fields)
        yield example

In [35]:
iter_test =  make_examples(train_sample, {"label": LABEL, "comment_text": TEXT})

In [39]:
ex = next(iter_test)
print(ex.label)
print(ex.comment_text)

3it [00:41, 17.31s/it]

0
["doesn't", 'ever', 'leave', 'the', 'airport', 'alaska', 'airlines', 'much', 'better']


In [69]:
torch.cuda.current_device()

0

In [71]:
torch.cuda.get_device_capability(device=0)

(3, 7)

In [136]:
torch.cuda.memory_allocated(device=0)

10752